# Investor - Flow of Funds - US

### Introduction:

Special thanks to: https://github.com/rgrp for sharing the dataset.

### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
import seaborn as sns

In [3]:
import requests as r

In [2]:
spark = SparkSession.builder.master("local[1]").appName("scores").getOrCreate()

23/01/02 20:30:00 WARN Utils: Your hostname, xkeyscore resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp0s20f3)
23/01/02 20:30:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 20:30:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/02 20:30:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/02 20:30:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/02 20:30:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/02 20:30:01 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/01/02 20:30:01 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/01/02 20:30:01 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv). 

In [8]:
URL = "https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv"
req = r.get(URL)

with open('weekly.csv','w+') as f:
    f.write(req.text)
    

### Step 3. Assign it to a variable called 

In [9]:
data = spark.read.csv('weekly.csv', inferSchema=True, header=True)

In [10]:
data.show(10)

+-------------------+------------+---------------+------------+------+----------+------------+--------------+-----+
|               Date|Total Equity|Domestic Equity|World Equity|Hybrid|Total Bond|Taxable Bond|Municipal Bond|Total|
+-------------------+------------+---------------+------------+------+----------+------------+--------------+-----+
|2012-12-05 00:00:00|       -7426|          -6060|       -1367|   -74|      5317|        4210|          1107|-2183|
|2012-12-12 00:00:00|       -8783|          -7520|       -1263|   123|      1818|        1598|           219|-6842|
|2012-12-19 00:00:00|       -5496|          -5470|         -26|   -73|       103|        3472|         -3369|-5466|
|2012-12-26 00:00:00|       -4451|          -4076|        -375|   550|      2610|        3333|          -722|-1291|
|2013-01-02 00:00:00|      -11156|          -9622|       -1533|  -158|      2383|        2103|           280|-8931|
|2013-01-09 00:00:00|       14817|           7995|        6821|  2888|  

In [11]:
data.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Total Equity: integer (nullable = true)
 |-- Domestic Equity: integer (nullable = true)
 |-- World Equity: integer (nullable = true)
 |-- Hybrid: integer (nullable = true)
 |-- Total Bond: integer (nullable = true)
 |-- Taxable Bond: integer (nullable = true)
 |-- Municipal Bond: integer (nullable = true)
 |-- Total: integer (nullable = true)



### Step 4.  What is the frequency of the dataset?

In [12]:
# weekly

### Step 5. Set the column Date as the index.

In [13]:
# not possible in Pyspark dataframes

### Step 6. What is the type of the index?

### Step 7. Set the index to a DatetimeIndex type

### Step 8.  Change the frequency to monthly, sum the values and assign it to monthly.

In [22]:
monthly = data.select(F.date_format('Date', 'yyyy-MM').alias('month'),'*').groupby('month').sum()
monthly.show(10)

+-------+-----------------+--------------------+-----------------+-----------+---------------+-----------------+-------------------+----------+
|  month|sum(Total Equity)|sum(Domestic Equity)|sum(World Equity)|sum(Hybrid)|sum(Total Bond)|sum(Taxable Bond)|sum(Municipal Bond)|sum(Total)|
+-------+-----------------+--------------------+-----------------+-----------+---------------+-----------------+-------------------+----------+
|2012-12|           -26156|              -23126|            -3031|        526|           9848|            12613|              -2765|    -15782|
|2015-02|             5508|                3509|             1999|       1752|           9099|             7443|               1656|     16359|
|2014-08|             1943|               -5508|             7452|       1885|           1897|            -1013|               2910|      5723|
|2014-09|            -2767|               -6596|             3829|       1599|           3984|             2479|               1504|    

### Step 9. You will notice that it filled the dataFrame with months that don't have any data with NaN. Let's drop these rows.

### Step 10. Good, now we have the monthly data. Now change the frequency to year.

In [23]:
yearly = data.select(F.date_format('Date', 'yyyy').alias('year'),'*').groupby('year').sum()
yearly.show(10)

+----+-----------------+--------------------+-----------------+-----------+---------------+-----------------+-------------------+----------+
|year|sum(Total Equity)|sum(Domestic Equity)|sum(World Equity)|sum(Hybrid)|sum(Total Bond)|sum(Taxable Bond)|sum(Municipal Bond)|sum(Total)|
+----+-----------------+--------------------+-----------------+-----------+---------------+-----------------+-------------------+----------+
|2012|           -26156|              -23126|            -3031|        526|           9848|            12613|              -2765|    -15782|
|2014|              330|              -44689|            45021|      19570|          59890|            44994|              14896|     79787|
|2013|             3661|               -1627|             5288|       2730|          12149|             9414|               2735|     18540|
|2015|            15049|              -10459|            25508|       7280|          26028|            17986|               8041|     48357|
+----+-------

### BONUS: Create your own question and answer it.